In [1]:
import os
os.system("sudo pip3 install findspark sql_magic")

256

In [2]:
# Use findspark package to connect Jupyter to Spark shell
import findspark
findspark.init('/usr/lib/spark')

ImportError: No module named 'findspark'

In [3]:
# Load SparkSession object
import pyspark
from pyspark.sql import SparkSession

# Load other libraries
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
import pandas as pd

In [16]:
# Set initial spark parameters to include Spark NLP package
pyspark_submit_args = '--packages JohnSnowLabs:spark-nlp:1.5.3 pyspark-shell'
# pyspark_submit_args = ' --jars /home/hadoop/spark_nlp_test/data/spark-nlp_2.11-1.5.3.jar pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
os.environ["PYTHONPATH"] = '/home/hadoop/spark_nlp_test/data/spark-nlp_2.11-1.5.3.jar'

In [17]:
# Initiate SparkSession as "spark"
spark = SparkSession\
    .builder\
    .getOrCreate()

In [18]:
import sys
import glob
sys.path.extend(glob.glob(os.path.join(os.path.expanduser("~"), ".ivy2/jars/*.jar")))

In [21]:
import sparknlp.annotator

AttributeError: 'module' object has no attribute 'annotator'

## Load Data

In [7]:
# Read tweet data
df = spark.read.csv(
    "s3n://2017edmfasatb/spark_nlp_test/metootweets.csv", 
    header = True, 
    inferSchema = True
)

In [9]:
# Preview schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- insertdate: string (nullable = true)
 |-- twitterhandle: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- hashtagsearched: string (nullable = true)
 |-- tweetid: string (nullable = true)
 |-- dateoftweet: string (nullable = true)
 |-- text: string (nullable = true)
 |-- lastcontactdate: string (nullable = true)
 |-- lasttimelinepull: string (nullable = true)
 |-- lasttimetweetsanalyzed: string (nullable = true)
 |-- numberoftweetsanalysed: string (nullable = true)
 |-- numberoftweetsabouthash: string (nullable = true)
 |-- actualtwitterdate: string (nullable = true)



In [8]:
df.limit(10).toPandas()

,id,insertdate,twitterhandle,followers,hashtagsearched,tweetid,dateoftweet,text,lastcontactdate,lasttimelinepull,lasttimetweetsanalyzed,numberoftweetsanalysed,numberoftweetsabouthash,actualtwitterdate
0,2365116,2018-02-08 16:38:00,comsatori,7216,metoo,961730576167288832,Thu Feb 08 22:36:52 +0000 2018,RT @IxAmandaDelgado: @Navegaciones @FelipeCald...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
1,2365115,2018-02-08 16:38:00,mchris4duke,9661,metoo,961730578398830592,Thu Feb 08 22:36:52 +0000 2018,RT @alexwitze: .@NSF will require institutions...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
2,2365114,2018-02-08 16:38:00,munakk,10,metoo,961730610824871936,Thu Feb 08 22:37:00 +0000 2018,Listening to the awesome feminist scholar Cynt...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
3,2365113,2018-02-08 16:38:00,masayakondo,1487,metoo,961730619393810432,Thu Feb 08 22:37:02 +0000 2018,??????????????????????????????????????????????...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
4,2365112,2018-02-08 16:38:00,eddydur,33,metoo,961730655506915328,Thu Feb 08 22:37:11 +0000 2018,"RT @AlbertoBernalLe: ?A ver, donde est?n todas...",2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
5,2365111,2018-02-08 16:38:00,cruz_madi,246,metoo,961730674364375040,Thu Feb 08 22:37:15 +0000 2018,"""RT @sassysanborn: We cant romanticize the sam...",2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
6,2365110,2018-02-08 16:38:00,tlsg99,4155,metoo,961730716349288448,Thu Feb 08 22:37:25 +0000 2018,RT @DefendEvropa: #120dB is a new initiative b...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
7,2365109,2018-02-08 16:38:00,ZeroOption4Shef,141,metoo,961730723794313217,Thu Feb 08 22:37:27 +0000 2018,RT @sarnjitflora: Very proud to become a White...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
8,2365108,2018-02-08 16:38:00,irishspy,19215,metoo,961730725908066304,Thu Feb 08 22:37:27 +0000 2018,#MeToo movement lawmaker investigated for sexu...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
9,2365107,2018-02-08 16:38:00,CanadasMajority,1114,metoo,961730743297654784,Thu Feb 08 22:37:31 +0000 2018,@DianeMariePosts @kwralex @gmbutts #GeraldButt...,2015-01-01 00:00:00,2015-01-01 00:00:00,2015-01-01 00:00:00,0,0,None
